# Azure Logic App

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [3]:
import os
from typing import Set
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from utils.user_logic_apps import AzureLogicAppTool, create_get_weather_function
from azure.ai.agents.models import ToolSet, FunctionTool, MessageRole

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [4]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Setup Logic App tool

In [5]:
# Extract subscription and resource group from the project scope
subscription_id = os.environ["SUBSCRIPTION_ID"]
resource_group = os.environ["RESOURCE_GROUP_NAME"]

# Logic App details
logic_app_name = "logicapp"
trigger_name = "When_a_HTTP_request_is_received"

# Create and initialize AzureLogicAppTool utility
logic_app_tool = AzureLogicAppTool(subscription_id, resource_group)
logic_app_tool.register_logic_app(logic_app_name, trigger_name)
print(f"Registered logic app '{logic_app_name}' with trigger '{trigger_name}'.")

# Create the specialized "get_weather_logic_app" function for your agent tools
get_weather_func = create_get_weather_function(logic_app_tool, logic_app_name)

# Prepare the function tools for the agent
functions_to_use: Set = {
    get_weather_func,  # This references the AzureLogicAppTool instance via closure
}
functions = FunctionTool(functions=functions_to_use)
toolset = ToolSet()
toolset.add(functions)

project_client.agents.enable_auto_function_calls(toolset)

Registered logic app 'logicapp' with trigger 'When_a_HTTP_request_is_received'.


## Create the agent

In [14]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-azurelogicapp-agent",  # Name of the agent
    instructions="You are a helpful agent that gets the weather. Provide answers in tabular format.",  # Instructions for the agent
    toolset=toolset
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_q4gK5aiIwLVv0ineHc8FOUpJ


## Create thread

In [15]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_AVWA3XKrXbSk3fLCNcVmhPub


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [12]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Run the agent

In [16]:
user_input = "What is the weather in London?"
run_agent(user_input)

Created message, ID: msg_5RxFMzGzJiGbA8nhD9dk4G5z
Run finished with status: completed
Agent Response: Here is the weather information for London:

| Weather Element       | Value                      |
|-----------------------|----------------------------|
| Condition             | Partly sunny               |
| Temperature           | 18.0°C                     |
| Feels Like            | 22.0°C                     |
| Dew Point             | 9.0°C                      |
| Humidity              | 58%                        |
| Barometric Pressure   | 1029.0 hPa                 |
| Visibility            | 10 km                      |
| Wind Direction        | 220°                       |
| Wind Speed            | 8.0 km/h                   |
| Wind Gust             | 14.0 km/h                  |
| UV Index              | 0.0 (Low)                  |
| Air Quality Index     | 34 (Good air quality)      |
| Primary Pollutant     | O₃ (0.0 ppb)               |
| Cloud Cover           | 59

In [17]:
user_input = "What is the weather in Chicago?"
run_agent(user_input)

Created message, ID: msg_43SnhCi6txThfti1110f8Qyu
Run finished with status: completed
Agent Response: Here is the weather information for Chicago:

| Weather Element       | Value                            |
|-----------------------|----------------------------------|
| Condition             | Mostly clear                     |
| Temperature           | 23.0°C                           |
| Feels Like            | 23.0°C                           |
| Dew Point             | 20.0°C                           |
| Humidity              | 83%                              |
| Barometric Pressure   | 1015.0 hPa                       |
| Visibility            | 16 km                            |
| Wind Direction        | 217°                             |
| Wind Speed            | 7.0 km/h                         |
| Wind Gust             | 10.0 km/h                        |
| UV Index              | 0.0 (Low)                        |
| Air Quality Index     | 87 (Moderate air quality)        

## Cleanup resources

In [ ]:
# Cleanup resources

# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
